


## Approach used:


    The following notebook contains the whole process of creating and comparing machine learning classifieres to identify the hate speech from textual input. I will be starting off with the data visualization, then I will preprocess the text annd then I will move for model training, tuning, the comparision betIen different models and then finally I will save our model in the disk for making predictions in future.
    
    
    Here are the steps that the project will be following to achieve the work that needs to be done:
    
   #### 1. Dataset loading:
    1) It started off with reading data set.  Made a single dataframe of data set. The data set will have two columns, one column will represent text and the other will represent the text type. 
    
    
   #### 2. Data visualization:
    2) It then did some data visualization that includes showing the data distribution, text information, wordclouds and histogram.
    
   #### 3. Data preprocessing:
    3) Data preprocessing plays an important role in NLP models. Since there are some chracters/words that can hinders the model performance.So our data processing includes removing speical characters, lemmatizing the words, replacing emojis with text.

####  4. Training ,comparing the models performance :
    4)  Then convert the text into numerical data using tf-idf vectorizer and trained our machine learning models on the vectorized data. 
    4a.For the evaluation of the classifier built the project used the metric accuracy that was tested on differnt folds using k-fold cross validation. And then I plot the  confusion matrix and an accuracy plot.

#### 5. Fine tuning and saving the model:
    5. Then fine tune one of the model and saved it in the disk using pickle so in future predictions can be made using the saved models.




## 1. Dataset loading:

In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
df= pd.read_csv("train.csv",encoding='latin-1')
df=df[['label','tIet']]
df.columns=["Target","text"]
df.head()



no_hate=df[df['Target']==0]
hate=df[df['Target']==1]




## 2. Data visualization:


In [2]:

### Pie chart showing the class distribution in data

import matplotlib.pyplot as plt

class_names = (df['Target'].unique()) ## This line will store the name of the labels, since in our case
## labels are stored in the form of numbers i.e 0 for non hate , 1 for hate speech
## So this variable will store only 0 and 1


## This variable will store the occurance of 0 and 1 in our data set
values_of_classes = list(df['Target'].value_counts())


colors = ['blue', 'yellow', 'brown', 'black']
patches, texts,per = plt.pie(values_of_classes,autopct='%0.5f%%', colors=colors, startangle=90)
plt.legend(patches, class_names, loc="best")


plt.title('Data Distribution for each label')
plt.show()


In [3]:
#Taken from
#https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words = 'english')

from wordcloud import WordCloud
no_hate['text'] = no_hate['text'].str.replace('@user',' ')

#Getting vectors for each word of text
words = vectorizer.fit_transform(no_hate.text)

#Summing the vectors across the rows
sum_words = words.sum(axis=0)

#Getting the frequency of each word
words_frequency = [(word, sum_words[0, i]) for word, i in vectorizer.vocabulary_.items()]

#Sorting the words with in ascending order
words_frequency = sorted(words_frequency, key = lambda x: x[1], reverse = True)

#Making a dataframe of the occurances
occurance = pd.DataFrame(words_frequency, columns=['words', 'occurance'])

occurance.head(30).plot(x='words', y='occurance', kind='bar', figsize=(12, 6), color = 'blue')
plt.title("- Top 30 - Most Frequently Occuring Words in non hate speech - Top 30")




Text(0.5, 1.0, '- Top 30 - Most Frequently Occuring Words in non hate speech - Top 30')

In [4]:
wordcloud = WordCloud( width = 1200, height = 1200,background_color = 'white').generate_from_frequencies(dict(words_frequency))

plt.figure(figsize=(10,8))
plt.imshow(wordcloud)
plt.title("WordCloud - Frequently Occuring Words in non hate speech", fontsize = 12)


Text(0.5, 1.0, 'WordCloud - Frequently Occuring Words in non hate speech')

In [5]:
## Replacing the word @user 
## Refernece : https://stackoverflow.com/questions/28986489/how-to-replace-text-in-a-column-of-a-pandas-dataframe

hate['text'] = hate['text'].str.replace('@user',' ')


## Getting vectors for each word of text
words = vectorizer.fit_transform(hate.text)
## Summing the vectors across the rows
sum_words = words.sum(axis=0)

## Getting the frequency of each word
words_frequency = [(word, sum_words[0, i]) for word, i in vectorizer.vocabulary_.items()]
## Sorting the words with in ascending order
words_frequency = sorted(words_frequency, key = lambda x: x[1], reverse = True)
## Making a dataframe of the occurances
occurance = pd.DataFrame(words_frequency, columns=['words', 'occurance'])

occurance.head(30).plot(x='words', y='occurance', kind='bar', figsize=(12, 6), color = 'blue')
plt.title("- Top 30 - Most Frequently Occuring Words in  hate speech ")


Text(0.5, 1.0, '- Top 30 - Most Frequently Occuring Words in  hate speech ')

In [6]:

wordcloud = WordCloud(background_color = 'white', width = 1200, height = 1200).generate_from_frequencies(dict(words_frequency))

plt.figure(figsize=(10,8))
plt.imshow(wordcloud)
plt.title("Frequently Occuring Words in hate speech", fontsize = 12)


# In[ ]:




Text(0.5, 1.0, 'Frequently Occuring Words in hate speech')

## 3. Data preprocessing:

In [7]:
import re as reg
import nltk
import time
import seaborn as sns
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.metrics import confusion_matrix
import pandas as pd
from nltk.stem.porter import PorterStemmer
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import KFold
import joblib as joblib
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup


In [8]:
## Shuffling the data frame 

## Reference : https://stackoverflow.com/questions/29576430/shuffle-dataframe-rows

from sklearn.utils import shuffle
df = shuffle(df)

In [9]:
## For processing the tIets the some parts are taken from this
## gitlab repository : https://gitlab.lipi.go.id/ndung/hpc/tree/master


def replacing_emoji(text_of_tIet):
    """
    This function will replace the emojis with suitable text in the tIets
    """
    # Laughing emojis -- :D, : D, :-D, xD, x-D, XD, X-D
    text_of_tIet = reg.sub(r'(:\s?D|:-D|x-?D|X-?D)', ' EMO_POS ', text_of_tIet)
    # Love emojis -- <3, :*
    text_of_tIet = reg.sub(r'(<3|:\*)', ' EMO_POS ', text_of_tIet)
    # Winking emojis -- ;-), ;), ;-D, ;D, (;,  (-;
    text_of_tIet = reg.sub(r'(;-?\)|;-?D|\(-?;)', ' EMO_POS ', text_of_tIet)
    # Smiling emojis -- :), : ), :-), (:, ( :, (-:, :')
    text_of_tIet = reg.sub(r'(:\s?\)|:-\)|\(\s?:|\(-:|:\'\))', ' EMO_POS ', text_of_tIet)
    # Sad emojis-- :-(, : (, :(, ):, )-:
    text_of_tIet = reg.sub(r'(:\s?\(|:-\(|\)\s?:|\)-:)', ' EMO_NEG ', text_of_tIet)
    # Cry emojis-- :,(, :'(, :"(
    text_of_tIet = reg.sub(r'(:,\(|:\'\(|:"\()', ' EMO_NEG ', text_of_tIet)
    return text_of_tIet




def preprocess_tIet_word(tIet_word):
    """ 
    This function will remove the repitition of words and ',- symbols from the text of tIets
    """
    # Remove punctuation
    tIet_word = tIet_word.strip('\'"?!,.():;')
    # Converting more than two letter repetitions to two letter
    # funnnnny --> funny
    tIet_word = reg.sub(r'(.)\1+', r'\1\1', tIet_word)
    # Remove - & '
    tIet_word = reg.sub(r'(-|\')', '', tIet_word)
    return tIet_word


def is_valid_tIet_word(tIet_word):
    ## Reference :https://stackoverflow.com/questions/27198984/regex-match-if-strings-starts-with-a-letter-or-number-and-then
    # Check if tIet_word begins with an alphabet
    return (reg.search(r'^[a-zA-Z][a-z0-9A-Z\._]*$', tIet_word) is not None)


def preprocess_text_of_tIet(text_of_tIet,use_stemmer=False):
    """
    This function will remove the URLS , mentioins , hashtags in the text
    """
    processed_text_of_tIet = []
    # Convert to loIr case
    text_of_tIet = text_of_tIet.loIr()
    # Replaces URLs with the tIet_word URL
    text_of_tIet = reg.sub(r'((www\.[\S]+)|(https?://[\S]+))', ' URL ', text_of_tIet)
    # Replace two+ dots with space
    text_of_tIet = reg.sub(r'\.{2,}', ' ', text_of_tIet)
    # Strip space, " and ' from text_of_tIet
    text_of_tIet = text_of_tIet.strip(' "\'')
    # Replace emojis with either EMO_POS or EMO_NEG
    text_of_tIet = replacing_emoji(text_of_tIet)
    # Replace @handle with the tIet_word USER_MENTION
    text_of_tIet = reg.sub(r'@[\S]+', 'USER_MENTION', text_of_tIet)
    # Replaces #hashtag with hashtag
    text_of_tIet = reg.sub(r'#(\S+)', r' \1 ', text_of_tIet)
    # Remove RT (retext_of_tIet)
    text_of_tIet = reg.sub(r'\brt\b', '', text_of_tIet)
    # Replace multiple spaces with a single space
    text_of_tIet = reg.sub(r'\s+', ' ', text_of_tIet)
    tIet_words = text_of_tIet.split()

    for tIet_word in tIet_words:
        tIet_word = preprocess_tIet_word(tIet_word)
        if is_valid_tIet_word(tIet_word):
            if use_stemmer:
                tIet_word = str(porter_stemmer.stem(tIet_word))
            processed_text_of_tIet.append(tIet_word)

    return ' '.join(processed_text_of_tIet)


def preprocess_csv(lines,use_stemmer=False):
    """
    This function will make a data frame for the processed data
    """
    total = len(lines)
    pre_processed=[]
    for i in range(total):
        pre_processed.append(preprocess_text_of_tIet(lines[i]))
        
    return pre_processed

## 4. Training ,comparing the models performance :

In [10]:
## Guide for performing cross validation was taken from this reference :
## https://my.oschina.net/Bettyty/blog/751627

def k_fold_cross_validation(training_columns, labels, vectorizer, classifier, name_of_model): 
 
    kf = KFold( shuffle=True,n_splits=10) # Initializing KFold object with 10 as k-value
    accuracies=[]
    training_columns_train_array = []
    labels_train_array = []
    training_columns_test_array = []
    labels_test_array = []
    iter = 0
    print("Applying K fold cv algorithm for :", (name_of_model))
    for training_indices, test_indices in kf.split(training_columns):
        iter += 1
        print("iter ", iter)
        training_columns_train_cv, labels_train_cv = training_columns.iloc[training_indices], labels.iloc[training_indices]
        training_columns_test_cv, labels_test_cv = training_columns.iloc[test_indices], labels.iloc[test_indices]
        
        # adding training score for  the given iteration
        training_columns_train_array.append(training_columns_train_cv) 
        
        # adding test score for iteration
        training_columns_test_array.append(training_columns_test_cv) 
        
         # adding training labels for  the given iteration
        labels_train_array.append(labels_train_cv) 
        
        # adding test labels for  the given iteration
        labels_test_array.append(labels_test_cv)  
        
        # learning vocabulary of training set
        vectorizer.fit(training_columns_train_cv) 
        training_columns_train_vectorizer_cv = vectorizer.transform(training_columns_train_cv)
        print("Shape of training training_columns: ", training_columns_train_vectorizer_cv.shape)
        training_columns_test_vectorizer_cv = vectorizer.transform(training_columns_test_cv)
        print("Shape of test training_columns: ", training_columns_test_vectorizer_cv.shape)
        classifier.fit(training_columns_train_vectorizer_cv.toarray(), labels_train_cv) 
        
        # Calculating accuracy
        score = classifier.score(training_columns_test_vectorizer_cv.toarray(), labels_test_cv)
        
        # adding k-fold  accuracy for each iter
        accuracies.append(score) 
    print("List of k-fold  accuracies for {}: ".format(name_of_model), accuracies)
    average_accuracy = np.mean(accuracies)
    print("Average k-fold  accuracy for {}: ".format(name_of_model), average_accuracy)
    print("Best k-fold  accuracy for {}: ".format(name_of_model), max(accuracies))
    
    # best k-fold  accuracy
    maximum_accuracy_index = accuracies.index(max(accuracies)) 
    
    # training training_columns corresponding to best k-fold  accuracy
    maximum_accuracy_training_columns_train = training_columns_train_array[maximum_accuracy_index] 
    
    # test training_columns corresponding to best k-fold  accuracy
    maximum_accuracy_training_columns_test = training_columns_test_array[maximum_accuracy_index] 
    
    # training labels corresponding to best k-fold  accuracy
    maximum_accuracy_labels_train = labels_train_array[maximum_accuracy_index] 
    
    # test labels corresponding to best k-fold  accuracy
    maximum_accuracy_labels_test = labels_test_array[maximum_accuracy_index] 

 

    return average_accuracy, maximum_accuracy_training_columns_train, maximum_accuracy_training_columns_test, maximum_accuracy_labels_train, maximum_accuracy_labels_test,accuracies

 
def evaluate(maximum_accuracy_training_columns_train, maximum_accuracy_training_columns_test, maximum_accuracy_labels_train, maximum_accuracy_labels_test, vectorizer, labels, clf, name_of_model): #### Creates Confusion matrix for SVC
    vectorizer.fit(maximum_accuracy_training_columns_train)
    maximum_accuracy_training_columns_train_vectorizer = vectorizer.transform(maximum_accuracy_training_columns_train)
    maximum_accuracy_training_columns_test_vectorizer = vectorizer.transform(maximum_accuracy_training_columns_test)
    clf.fit(maximum_accuracy_training_columns_train_vectorizer, maximum_accuracy_labels_train) 
    
    # Prediction on test training_columns
    labels_pred = clf.predict(maximum_accuracy_training_columns_test_vectorizer) 
    
    ## Plotting confusion matrix is taken from :
    ## https://medium.com/@dtuk81/confusion-matrix-visualization-fc31e3f30fea
    cf_matrix = confusion_matrix(maximum_accuracy_labels_test, labels_pred)
    d={ 0: 'Non Hate', 1: 'Hate'}
    sentiment_df = labels.drop_duplicates().sort_values()
    
    group_names=["Correct Non Hate","Incorrect Hate","Incorrect Non Hate","Correct Hate"]
    
    group_counts = ["{0:0.0f}".format(value) for value in
                cf_matrix.flatten()]
    group_percentages = ["{0:.2%}".format(value) for value in
                     cf_matrix.flatten()/np.sum(cf_matrix)]
    labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in zip(group_names,group_counts,group_percentages)]
    labels = np.asarray(labels).reshape(2,2)


    sns.heatmap(cf_matrix, annot=labels, fmt='', xticklabels=sentiment_df.values, yticklabels=sentiment_df.values)
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.title("Confusion Matrix (Best Accuracy) - {}".format(name_of_model))
    plt.show()
    return

In [11]:
df['normalized']=preprocess_csv(df['text'].values)

In [12]:
from sklearn.naive_bayes import MultinomialNB

data = df.normalized
targets = df.Target

# min_df=30 is a clever way of feature engineering
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=30, norm='l2', ngram_range=(1,3)) 

# NBC Model
NBC_clf = MultinomialNB()

# NBC cross-validation
NBC_average_acc, training_data_with_max_acc, test_data_with_max_acc, targets_with_max_acc_train, targets_with_max_acc_test,nbc_accuracies = k_fold_cross_validation(data, targets, tfidf, NBC_clf, "NBC") 

# NBC confusion matrix
evaluate(training_data_with_max_acc, test_data_with_max_acc, targets_with_max_acc_train, targets_with_max_acc_test, tfidf, targets, NBC_clf, "NBC") 

plt.plot(nbc_accuracies,color='r',label="NBC")
plt.legend(loc="best")

## Setting x-label of fig 
plt.xlabel("Folds") 

## Setting y-label of fig 
plt.ylabel("Accuracy"); 

## Setting subtitle of fig
plt.title("Score across each fold") 


Applying K fold cv algorithm for : NBC
iter  1
Shape of training training_columns:  (28923, 2197)
Shape of test training_columns:  (3214, 2197)
iter  2
Shape of training training_columns:  (28923, 2217)
Shape of test training_columns:  (3214, 2217)
iter  3
Shape of training training_columns:  (28923, 2197)
Shape of test training_columns:  (3214, 2197)
iter  4
Shape of training training_columns:  (28923, 2230)
Shape of test training_columns:  (3214, 2230)
iter  5
Shape of training training_columns:  (28923, 2217)
Shape of test training_columns:  (3214, 2217)
iter  6
Shape of training training_columns:  (28923, 2207)
Shape of test training_columns:  (3214, 2207)
iter  7
Shape of training training_columns:  (28923, 2198)
Shape of test training_columns:  (3214, 2198)
iter  8
Shape of training training_columns:  (28924, 2212)
Shape of test training_columns:  (3213, 2212)
iter  9
Shape of training training_columns:  (28924, 2209)
Shape of test training_columns:  (3213, 2209)
iter  10
Shape o

Text(0.5, 1.0, 'Score across each fold')

In [13]:
#ref: https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.ComplementNB.html#sklearn.naive_bayes.ComplementNB
        

import numpy as np
from sklearn.naive_bayes import ComplementNB

CNB_rng = np.random.RandomState(1)

xlabel = CNB_rng.randint(5, size=(6, 100))
ylabel = np.array([1, 2, 3, 4, 5, 6])

data = df.normalized
targets = df.Target

# min_df=30 is a clever way of feature engineering
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=30, norm='l2', ngram_range=(1,3)) 

# CNB Model
B_clf = ComplementNB() 
#B_clf.fit(xlabel, ylabel, sample_Iight=None)
B_clf.fit(xlabel, ylabel)

# NBC cross-validation
CNB_average_acc, training_data_with_max_acc, test_data_with_max_acc, targets_with_max_acc_train, targets_with_max_acc_test,bnc_accuracies = k_fold_cross_validation(data, targets, tfidf, B_clf, "ComplementNB") 

# NBC confusion matrix
evaluate(training_data_with_max_acc, test_data_with_max_acc, targets_with_max_acc_train, targets_with_max_acc_test, tfidf, targets, B_clf, "ComplementNB") 


plt.plot(bnc_accuracies,color='r',label="CNB")
plt.legend(loc="best")

plt.title("Score across each fold") ## Setting subtitle of fig 1
plt.xlabel("Folds") 
plt.ylabel("Accuracy"); ## Setting y-label of fig 1



Applying K fold cv algorithm for : ComplementNB
iter  1
Shape of training training_columns:  (28923, 2207)
Shape of test training_columns:  (3214, 2207)
iter  2
Shape of training training_columns:  (28923, 2208)
Shape of test training_columns:  (3214, 2208)
iter  3
Shape of training training_columns:  (28923, 2206)
Shape of test training_columns:  (3214, 2206)
iter  4
Shape of training training_columns:  (28923, 2215)
Shape of test training_columns:  (3214, 2215)
iter  5
Shape of training training_columns:  (28923, 2189)
Shape of test training_columns:  (3214, 2189)
iter  6
Shape of training training_columns:  (28923, 2223)
Shape of test training_columns:  (3214, 2223)
iter  7
Shape of training training_columns:  (28923, 2211)
Shape of test training_columns:  (3214, 2211)
iter  8
Shape of training training_columns:  (28924, 2205)
Shape of test training_columns:  (3213, 2205)
iter  9
Shape of training training_columns:  (28924, 2221)
Shape of test training_columns:  (3213, 2221)
iter  1

In [14]:
#ref: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix ##


LR_clf = LogisticRegression(random_state=0).fit(xlabel, ylabel)
LR_clf.predict(xlabel[:2, :])
LR_clf.predict_proba(xlabel[:2, :])



LR_average_acc, training_data_with_max_acc, test_data_with_max_acc, targets_with_max_acc_train, targets_with_max_acc_test,lr_accuracies = k_fold_cross_validation(data, targets, tfidf, LR_clf, "LR") # cross-validation
evaluate(training_data_with_max_acc, test_data_with_max_acc, targets_with_max_acc_train, targets_with_max_acc_test, tfidf, targets, LR_clf, "LR") # LR confusion matrix

## Comparision through graphs

plt.plot(lr_accuracies,color='r',label="LR")
plt.legend(loc="best")

plt.title("Score across each fold") ## Setting subtitle of fig 1
plt.xlabel("Folds") 
plt.ylabel("Accuracy"); ## Setting y-label of fig 1


print("Coeficient results")
print(LR_clf.coef_)

print("Intercepts results")
print(LR_clf.intercept_)







Applying K fold cv algorithm for : LR
iter  1
Shape of training training_columns:  (28923, 2207)
Shape of test training_columns:  (3214, 2207)
iter  2
Shape of training training_columns:  (28923, 2207)
Shape of test training_columns:  (3214, 2207)
iter  3
Shape of training training_columns:  (28923, 2207)
Shape of test training_columns:  (3214, 2207)
iter  4
Shape of training training_columns:  (28923, 2207)
Shape of test training_columns:  (3214, 2207)
iter  5
Shape of training training_columns:  (28923, 2209)
Shape of test training_columns:  (3214, 2209)
iter  6
Shape of training training_columns:  (28923, 2214)
Shape of test training_columns:  (3214, 2214)
iter  7
Shape of training training_columns:  (28923, 2215)
Shape of test training_columns:  (3214, 2215)
iter  8
Shape of training training_columns:  (28924, 2220)
Shape of test training_columns:  (3213, 2220)
iter  9
Shape of training training_columns:  (28924, 2207)
Shape of test training_columns:  (3213, 2207)
iter  10
Shape of

Coeficient results
[[-0.27777492 -0.25761044  1.49099071 ... -0.1400604  -0.45767266
  -0.4514109 ]]
Intercepts results
[-3.12246513]


In [15]:
#Ref: https://scikit-learn.org/stable/modules/tree.html
#ref: https://scikit-learn.org/stable/auto_examples/datasets/plot_iris_dataset.html

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix



DT_clf = DecisionTreeClassifier(random_state=0, max_depth=4)
DT = DT_clf.fit(xlabel, ylabel) #add 1
DT_average_acc, training_data_with_max_acc, test_data_with_max_acc, targets_with_max_acc_train, targets_with_max_acc_test,dt_accuracies = k_fold_cross_validation(data, targets, tfidf, DT_clf, "DT") # cross-validation

evaluate(training_data_with_max_acc, test_data_with_max_acc, targets_with_max_acc_train, targets_with_max_acc_test, tfidf, targets, DT_clf, "DT") # LR confusion matrix

## Comparision through graphs



plt.plot(dt_accuracies,color='r',label="Decision Tree")
plt.legend(loc="best")

plt.title("Score across each fold") ## Setting subtitle of fig 1
plt.xlabel("Folds") 
plt.ylabel("Accuracy"); ## Setting y-label of fig 1

 

Applying K fold cv algorithm for : DT
iter  1
Shape of training training_columns:  (28923, 2198)
Shape of test training_columns:  (3214, 2198)
iter  2
Shape of training training_columns:  (28923, 2196)
Shape of test training_columns:  (3214, 2196)
iter  3
Shape of training training_columns:  (28923, 2207)
Shape of test training_columns:  (3214, 2207)
iter  4
Shape of training training_columns:  (28923, 2211)
Shape of test training_columns:  (3214, 2211)
iter  5
Shape of training training_columns:  (28923, 2229)
Shape of test training_columns:  (3214, 2229)
iter  6
Shape of training training_columns:  (28923, 2208)
Shape of test training_columns:  (3214, 2208)
iter  7
Shape of training training_columns:  (28923, 2211)
Shape of test training_columns:  (3214, 2211)
iter  8
Shape of training training_columns:  (28924, 2220)
Shape of test training_columns:  (3213, 2220)
iter  9
Shape of training training_columns:  (28924, 2199)
Shape of test training_columns:  (3213, 2199)
iter  10
Shape of

In [16]:
## argument has accuracy values
plt.plot(nbc_accuracies,color='r',label="MNB")
plt.plot(bnc_accuracies,color='g',label="CNB")
plt.plot(lr_accuracies,color='b',label="LR")
plt.plot(dt_accuracies,color='m',label="DT")
plt.legend(loc="best")

plt.title("Score across each fold") ## Setting subtitle of fig 1
plt.xlabel("Folds") #Setting label info
plt.ylabel("Accuracy"); ## Setting y-label of fig 1

## 5. Fine tuning and saving the model:


In [17]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import reciprocal, uniform
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(tfidf.fit_transform(data), targets, test_size=0.1, random_state=101)

## Hyper-parametertuning
param_distributions = { "alpha": uniform(1, 10), "fit_prior":[True,False]}
rnd_search_cv = RandomizedSearchCV(NBC_clf, param_distributions, n_iter=10, verbose=2, cv=3)
rnd_search_cv.fit(X_train, y_train)



rnd_search_cv.best_estimator_

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] alpha=7.073450472985677, fit_prior=True .........................
[CV] .......... alpha=7.073450472985677, fit_prior=True, total=   0.0s
[CV] alpha=7.073450472985677, fit_prior=True .........................
[CV] .......... alpha=7.073450472985677, fit_prior=True, total=   0.0s
[CV] alpha=7.073450472985677, fit_prior=True .........................
[CV] .......... alpha=7.073450472985677, fit_prior=True, total=   0.0s
[CV] alpha=7.981346098012411, fit_prior=False ........................
[CV] ......... alpha=7.981346098012411, fit_prior=False, total=   0.0s
[CV] alpha=7.981346098012411, fit_prior=False ........................
[CV] ......... alpha=7.981346098012411, fit_prior=False, total=   0.0s
[CV] alpha=7.981346098012411, fit_prior=False ........................
[CV] ......... alpha=7.981346098012411, fit_prior=False, total=   0.0s
[CV] alpha=10.881847157275407, fit_prior=True ........................
[CV] ......... a

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV] ......... alpha=10.36232794845273, fit_prior=False, total=   0.0s
[CV] alpha=10.36232794845273, fit_prior=False ........................
[CV] ......... alpha=10.36232794845273, fit_prior=False, total=   0.0s
[CV] alpha=1.7530303607303168, fit_prior=True ........................
[CV] ......... alpha=1.7530303607303168, fit_prior=True, total=   0.0s
[CV] alpha=1.7530303607303168, fit_prior=True ........................
[CV] ......... alpha=1.7530303607303168, fit_prior=True, total=   0.0s
[CV] alpha=1.7530303607303168, fit_prior=True ........................
[CV] ......... alpha=1.7530303607303168, fit_prior=True, total=   0.0s
[CV] alpha=2.491098154819083, fit_prior=False ........................
[CV] ......... alpha=2.491098154819083, fit_prior=False, total=   0.0s
[CV] alpha=2.491098154819083, fit_prior=False ........................
[CV] ......... alpha=2.491098154819083, fit_prior=False, total=   0.0s
[CV] alpha=2.491098154819083, fit_prior=False ........................
[CV] .

[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    0.2s finished


MultinomialNB(alpha=1.7530303607303168)

In [18]:
rnd_search_cv.best_estimator_.fit(X_train, y_train)

MultinomialNB(alpha=1.7530303607303168)

In [19]:
import pickle
# now you can save it to a file
with open('hate_nohate_model.pkl', 'wb') as f:
    pickle.dump(rnd_search_cv.best_estimator_, f)